In [1]:
from gurobipy import *
import math
import random
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from shapely.geometry import Point, LineString
from shapely.geometry import Polygon
from statsmodels.stats.weightstats import DescrStatsW

In [2]:
#carrega alunos das subsecções de Ourém e matriz de distâncias
df_alunos_subseccao = pd.read_csv('./df_alunos_subseccao_2020.csv', dtype={'Unnamed: 0':str})
df_alunos_subseccao.index= df_alunos_subseccao['Unnamed: 0']
distanceMatrix = pd.read_csv('./00_Base_de_dados/Distancias/distancematrix-1421.csv', dtype={'to':str, 'origin':str})
distanceMatrix['Freg_origin'] = distanceMatrix.loc[:,'origin'].astype(str).str[0:6]
distanceMatrix['Freg_destino'] = distanceMatrix.loc[:,'to'].astype(str).str[0:6]
# df['new_col'] = df['First'].astype(str).str[0]
distanceMatrix.real_distance = round(distanceMatrix.real_distance/1000, 3)

In [3]:
#transforma distâncias em custos
distanceMatrix['custo'] = round((distanceMatrix.real_distance**2)*2*0.36*180*10,0)
#elimina subsecções sem alunos (não podem ser os residuais porque alguns têm alunos)
c_alunos = df_alunos_subseccao[df_alunos_subseccao['1_CEB_Proj_2020']>0].index
distanceMatrix_sres = distanceMatrix[distanceMatrix.to.isin(c_alunos)&distanceMatrix.origin.isin(c_alunos)]

In [4]:
dist_max = 17
Matriz_reduzida=distanceMatrix_sres[distanceMatrix_sres.real_distance <= dist_max]
# Matriz_reduzida = distanceMatrix_sres[(distanceMatrix_sres.Freg_origin.isin(['142105','142114']))& 
#                     (distanceMatrix_sres.Freg_destino.isin(['142105','142114']))]
# Matriz_reduzida = distanceMatrix_sres[(distanceMatrix_sres.Freg_origin=='142106') & (distanceMatrix_sres.Freg_destino=='142106')]

In [5]:
# capacidades = {21: (914650, 52752), 40: (1003910, 38540), 80: (1191826, 27500), 
#                140: (1473700, 20940), 210: (1802553,17190), 300: (2225364, 14450)}
capacidades = {21: (914650, 52752), 55: (1074379, 33010), 120: (1379742, 22570), 
               200: (1755574,17600), 300: (2225364, 14450)}

In [6]:
l = list(zip(Matriz_reduzida.origin, Matriz_reduzida.to, Matriz_reduzida.real_distance))
d_dist = {}
for (i,j,k) in l:
    d_dist[i,j] = k

In [7]:
alunos_por_subs  = pd.DataFrame(index=Matriz_reduzida.to.unique())
alunos_por_subs = alunos_por_subs.join(df_alunos_subseccao['1_CEB_Proj_2020'])
alunos_por_subs['1_CEB_Proj_2020'] = alunos_por_subs['1_CEB_Proj_2020'].fillna(0)
tuple_alunos_por_subs = dict(zip(alunos_por_subs.index, alunos_por_subs['1_CEB_Proj_2020']))
localizacoes = Matriz_reduzida.to.unique()
alunos_por_subs.sum()

1_CEB_Proj_2020    1345.02368
dtype: float64

In [8]:
lista = list(zip(Matriz_reduzida.origin, Matriz_reduzida.to, Matriz_reduzida.custo))
d_custos_dist = {}
for i, k, l  in lista: 
    d_custos_dist[i,k] = l

In [9]:
d_custos_instal={}
for i in localizacoes:
    for j in capacidades: 
        d_custos_instal [i, j] =  capacidades[j]
# d_custos_instal

In [10]:
l_orig_dest_capacid = []
l_tuplas  = list(zip(Matriz_reduzida.origin, Matriz_reduzida.to))

for i,j in l_tuplas:
    for k in capacidades:
        l_orig_dest_capacid.append((i,j,k))

In [11]:
m = Model('escolas')
m.reset()
#variável de decisão do número de alunos alocados de cada i a cada j, para cada tipo de capacidade
x = {}
x = m.addVars(l_orig_dest_capacid, lb=0.0, ub=28.0, vtype='C')

#variável de decisão abertura de escola 
y = {}
for j in localizacoes:
    for k in capacidades:
        y[(j,k)] = m.addVar(lb=0, vtype='B', name=f'escola_em_{j}_com_k={k}')

Using license file C:\Users\jan\gurobi.lic
Academic license - for non-commercial use only
Discarded solution information


In [12]:
#restrição que garante que todos os alunos são alocados
for i in tuple_alunos_por_subs:    
    m.addConstr(quicksum(x[i,j,k] for j in Matriz_reduzida.to[Matriz_reduzida.origin==i] for k in capacidades) == tuple_alunos_por_subs[i], f'alunos_alocados_{i}_{j}_{k}')

#restrição que garante que a alocação é feita para escolas abertas
for (i,j,k) in x:
    m.addConstr(x[i,j,k] <= tuple_alunos_por_subs[i]*y[j,k], f'alocacao_escolas_abertas_{i}_{j}_{k}')
    
#restrição que garante que a alocação respeita as capacidade das escolas
for j, k in y:
    m.addConstr(quicksum(x[i,j,k] for i in Matriz_reduzida.origin[Matriz_reduzida.to==j]) <= [j, k][1], f'capacidade_{k}_escola_{j}')

#restrição que garante que somente uma escola de cada dimensão é construída em cada local
for i in localizacoes:
    m.addConstr(quicksum(y[(i,c)] for c in capacidades) <= 1)

#restrição que garante que as escolas localizadas não possam ter menos de 20% de alunos do que a sua capacidade
for j, k in y:
    m.addConstr((quicksum(x[i,j,k] for i in Matriz_reduzida.origin[Matriz_reduzida.to==j]) - [j,k][1]*y[j,k]) >= [j,k][1]*y[j,k]*-0.20, f'restr_alocados_{j}')

#restrição que garante que nenhuma escola tem menos do que 21 alunos
for j, k in y:
    m.addConstr(quicksum(x[i,j,k] for i in Matriz_reduzida.origin[Matriz_reduzida.to==j]) >= 21*y[j,k], f'minimo_alocados_{j}')

m.setObjective(quicksum(x[i,j,k]*d_custos_dist[i,j]+x[i,j,k]*capacidades[k][1] for i,j,k in x) + quicksum(y[j]*d_custos_instal[j][0] for j in y), GRB.MINIMIZE)
m.setParam('TimeLimit', 1500*60)
m.setParam(GRB.Param.Method, 3)
m.update()

Changed value of parameter TimeLimit to 90000.0
   Prev: inf  Min: 0.0  Max: inf  Default: inf
Changed value of parameter Method to 3
   Prev: -1  Min: -1  Max: 5  Default: -1


In [13]:
m.optimize()

Gurobi Optimizer version 9.0.0 build v9.0.0rc2 (win64)
Optimize a model with 3001983 rows, 2990895 columns and 17931510 nonzeros
Model fingerprint: 0x6539550d
Variable types: 2986275 continuous, 4620 integer (4620 binary)
Coefficient statistics:
  Matrix range     [5e-01, 2e+02]
  Objective range  [1e+04, 2e+06]
  Bounds range     [1e+00, 3e+01]
  RHS range        [5e-01, 3e+02]
Presolve removed 4018 rows and 3960 columns (presolve time = 9s) ...
Presolve removed 4018 rows and 3960 columns (presolve time = 12s) ...
Presolve removed 5650 rows and 3960 columns (presolve time = 15s) ...
Presolve removed 5650 rows and 3960 columns (presolve time = 20s) ...
Presolve removed 5650 rows and 3960 columns (presolve time = 25s) ...
Presolve removed 5650 rows and 3960 columns (presolve time = 30s) ...
Presolve removed 6475 rows and 3960 columns (presolve time = 37s) ...
Presolve removed 6475 rows and 3960 columns (presolve time = 42s) ...
Presolve removed 7300 rows and 3960 columns (presolve time 

Thread count was 4 (of 4 available processors)

Solution count 2: 5.51259e+07 5.51317e+07 

Optimal solution found (tolerance 1.00e-04)
Best objective 5.512592882969e+07, best bound 5.512572498722e+07, gap 0.0004%


In [14]:
d_escolas_localizadas = {}
for j, l in y:
    if y[j,l].X > 0e-4:
        print(f'escola {j, l}: ', sum (x[i,j,l].X for i in Matriz_reduzida.origin[Matriz_reduzida.to==j]))
        d_escolas_localizadas [j] = l, sum (x[i,j,l].X for i in Matriz_reduzida.origin[Matriz_reduzida.to==j])

d={}
for i in x: 
    if x[i].X > 0.001:
        d[i]= x[i].X
sum(d.values())

escola ('14210500217', 55):  43.99999999999999
escola ('14211400411', 120):  96.00000000000001
escola ('14211300317', 120):  97.6220755673987
escola ('14210900101', 120):  104.32412017760056
escola ('14211100703', 200):  166.25742136694254
escola ('14211501005', 300):  299.99999999999994
escola ('14211200301', 120):  96.06922127603855
escola ('14211800220', 55):  44.0
escola ('14210600804', 300):  300.0
escola ('14210601605', 120):  96.75084161201896


1345.0236800000014

In [15]:
alunos_alocados = {}
for (i,j,k) in d:
    if x[(i,j,k)].X > 0.001:
        alunos_alocados [i,j] = i, j, d[i,j,k], d_dist[i,j]
    
df_alunos_alocados = pd.DataFrame.from_dict(alunos_alocados, orient='index', columns=[ 'SubSection', 'School', 'alunos_aloc', 'distancia',])
# df_alunos_alocados.sum(axis=0)
df_alunos_alocados['prod_dist_alun'] = df_alunos_alocados.alunos_aloc*df_alunos_alocados.distancia
df_alunos_alocados.prod_dist_alun.sum()/df_alunos_alocados.alunos_aloc.sum()

2.4447259853531906

In [16]:
# df_alunos_alocados_c_perc[df_alunos_alocados_c_perc.distancia<=1500].sum()

In [17]:
def Adicionar_percenatens (df_gini):
    df_gini = df_gini.sort_values(by= 'distancia')
    df_gini['percent_dist'] =  df_gini.prod_dist_alun / df_gini.prod_dist_alun.sum()
    df_gini['percent_alunos'] =  df_gini.alunos_aloc / df_gini.alunos_aloc.sum()
    df_gini['Dist_cumsum'] = df_gini.prod_dist_alun.cumsum()/df_gini.prod_dist_alun.sum()
    df_gini['Alunos_cumsum'] = df_gini.alunos_aloc.cumsum()/df_gini.alunos_aloc.sum()
    return(df_gini)
df_alunos_alocados_c_perc = Adicionar_percenatens(df_alunos_alocados)

In [18]:
df_alunos_alocados_c_perc.to_csv('alunos_alocados_2020_DISTANCIA_QUADRADO.csv')